In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Dataset 1. Covid Tracking Project Dataset
- this dataset is downloaded from [CovidTrackingProject](https://covidtracking.com)
- we have uploaded this dataset to GitHub as part of the codes

### Dataset Structure

In [2]:
df = pd.read_csv('all-states-history.csv')
print(df.shape)
df.head()

(13719, 42)


,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2020-11-02,AK,A,84.0,84.0,1,NaN,NaN,NaN,97.0,...,620170.0,15963,NaN,NaN,NaN,NaN,NaN,0,620170.0,15963
1,2020-11-02,AL,A,2973.0,2767.0,0,206.0,20450.0,20450.0,967.0,...,1356420.0,3840,NaN,NaN,64932.0,NaN,NaN,0,1356420.0,3840
2,2020-11-02,AR,A+,1985.0,1817.0,60,168.0,7110.0,7110.0,672.0,...,1363429.0,16457,NaN,21856.0,NaN,62842.0,NaN,0,1363429.0,16457
3,2020-11-02,AS,D,0.0,NaN,0,NaN,NaN,NaN,NaN,...,1768.0,152,NaN,NaN,NaN,NaN,NaN,0,1768.0,152
4,2020-11-02,AZ,A+,5982.0,5664.0,1,318.0,21573.0,21573.0,918.0,...,1792602.0,6835,320952.0,NaN,NaN,NaN,1792602.0,6835,NaN,0


In [3]:
df.drop('dataQualityGrade', axis=1, inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13719 entries, 0 to 13718
Data columns (total 41 columns):
date                                13719 non-null object
state                               13719 non-null object
death                               12901 non-null float64
deathConfirmed                      5728 non-null float64
deathIncrease                       13719 non-null int64
deathProbable                       4186 non-null float64
hospitalized                        7923 non-null float64
hospitalizedCumulative              7923 non-null float64
hospitalizedCurrently               10712 non-null float64
hospitalizedIncrease                13719 non-null int64
inIcuCumulative                     2313 non-null float64
inIcuCurrently                      6268 non-null float64
negative                            13413 non-null float64
negativeIncrease                    13719 non-null int64
negativeTestsAntibody               790 non-null float64
negativeTestsPeopleAnt

In [5]:
# sum of null values in each column; sort them in order
isnull_sum = df.isnull().sum()
isnull_sum.sort_values()

date                                    0
totalTestsPeopleViralIncrease           0
totalTestResultsIncrease                0
totalTestEncountersViralIncrease        0
positiveScore                           0
positiveIncrease                        0
negativeIncrease                        0
hospitalizedIncrease                    0
totalTestsViralIncrease                 0
state                                   0
deathIncrease                           0
totalTestResults                       35
positive                              133
negative                              306
death                                 818
hospitalizedCurrently                3007
positiveCasesViral                   3168
recovered                            4054
totalTestsViral                      5293
hospitalizedCumulative               5796
hospitalized                         5796
inIcuCurrently                       7451
deathConfirmed                       7991
totalTestsPeopleViral             

#### Conclusion:
- coulmns named "XXXIncrease" should be dropped
- totalestResults, positive, negative, hospitalizedCurrently, positiveCasesViral, recovered are acceptable to use
- ICU & ventilator need external datasets

Following codes were trying to select "good" states with ample data;  But we decided to use all states' data later on, hence we no longer need to select states.

In [7]:
# group by states; sum of null values of the whole state in each column
#null_by_state = df.groupby('state').apply(lambda x: x.isnull().sum())
#null_by_state['death'].shape

(56,)

In [9]:
# select states by constraints on 4 columns first (the 4 columns have the fewest null values on whole dataset)
#select1 = null_by_state[(null_by_state['hospitalizedCurrently']<50) 
            #& (null_by_state['positive']<50) 
            #& (null_by_state['negative']<50) 
            #& (null_by_state['totalTestResults']<50)
            #& (null_by_state['recovered']<50)]

In [6]:
null_by_date = df.groupby('date').apply(lambda x: x.isnull().sum())
print(null_by_date['death'].shape)
print(null_by_date['death'][:1], null_by_date['death'][-1:])

(286,)
date
2020-01-22    2
Name: death, dtype: int64 date
2020-11-02    0
Name: death, dtype: int64


#### Conclusion:
- this dataset covers data from 2020-01-22 to 2020-11-02 for every state in U.S

### Preprocessing specific columns

In [7]:
## transform to 'datetime' format data
datetime = pd.to_datetime(df.iloc[:, 0], format='%Y-%m-%d')
df.insert(0, 'datetime', datetime)
df.drop(['date'], axis=1, inplace=True)

In [8]:
print(df.shape)
df.head()

(13719, 41)


,datetime,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2020-11-02,AK,84.0,84.0,1,NaN,NaN,NaN,97.0,0,...,620170.0,15963,NaN,NaN,NaN,NaN,NaN,0,620170.0,15963
1,2020-11-02,AL,2973.0,2767.0,0,206.0,20450.0,20450.0,967.0,0,...,1356420.0,3840,NaN,NaN,64932.0,NaN,NaN,0,1356420.0,3840
2,2020-11-02,AR,1985.0,1817.0,60,168.0,7110.0,7110.0,672.0,55,...,1363429.0,16457,NaN,21856.0,NaN,62842.0,NaN,0,1363429.0,16457
3,2020-11-02,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,1768.0,152,NaN,NaN,NaN,NaN,NaN,0,1768.0,152
4,2020-11-02,AZ,5982.0,5664.0,1,318.0,21573.0,21573.0,918.0,15,...,1792602.0,6835,320952.0,NaN,NaN,NaN,1792602.0,6835,NaN,0


In [9]:
us_state_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',  ##
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',  ##
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',  ##
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',  ##
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',  ##
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [10]:
## transform state name abbreviation to full name
location_name = []
for i in df.loc[:, 'state']:
    location_name.append(us_state_abbrev[i])
df.insert(1, 'location_name', location_name)
df.drop(['state'], axis=1, inplace=True)
print(df.shape)
df.head()

(13719, 41)


,datetime,location_name,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2020-11-02,Alaska,84.0,84.0,1,NaN,NaN,NaN,97.0,0,...,620170.0,15963,NaN,NaN,NaN,NaN,NaN,0,620170.0,15963
1,2020-11-02,Alabama,2973.0,2767.0,0,206.0,20450.0,20450.0,967.0,0,...,1356420.0,3840,NaN,NaN,64932.0,NaN,NaN,0,1356420.0,3840
2,2020-11-02,Arkansas,1985.0,1817.0,60,168.0,7110.0,7110.0,672.0,55,...,1363429.0,16457,NaN,21856.0,NaN,62842.0,NaN,0,1363429.0,16457
3,2020-11-02,American Samoa,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,1768.0,152,NaN,NaN,NaN,NaN,NaN,0,1768.0,152
4,2020-11-02,Arizona,5982.0,5664.0,1,318.0,21573.0,21573.0,918.0,15,...,1792602.0,6835,320952.0,NaN,NaN,NaN,1792602.0,6835,NaN,0


In [11]:
## select columns with acceptable nulls
## *inIcuCurrently and onVetilatorCurrently still have a lot of null
features = ['datetime',
 'location_name',
 'death',
 'hospitalizedCurrently',
 'inIcuCurrently',
 'negative',
 'onVentilatorCurrently',
 'positive',
 'recovered',
 'totalTestResults']
df_clean = df.loc[:, features]
print(df_clean.shape)
df_clean.head()

(13719, 10)


,datetime,location_name,death,hospitalizedCurrently,inIcuCurrently,negative,onVentilatorCurrently,positive,recovered,totalTestResults
0,2020-11-02,Alaska,84.0,97.0,NaN,603113.0,6.0,17057.0,7110.0,620170.0
1,2020-11-02,Alabama,2973.0,967.0,NaN,1190436.0,NaN,194892.0,81005.0,1356420.0
2,2020-11-02,Arkansas,1985.0,672.0,251.0,1258726.0,112.0,113641.0,101507.0,1363429.0
3,2020-11-02,American Samoa,0.0,NaN,NaN,1768.0,NaN,0.0,NaN,1768.0
4,2020-11-02,Arizona,5982.0,918.0,231.0,1550382.0,120.0,248139.0,41589.0,1792602.0


# Dataset 2. Hospitalization data
- this dataset is downloaded from [HealthData](https://covid19.healthdata.org/united-states-of-america?view=total-deaths&tab=trend)
- we didn't upload this dataset on GitHub because its size reached maximum

### Dataset structure

In [21]:
df_hospital = pd.read_csv('hospitalization_all_locs.csv')
print(df_hospital.shape)
df_hospital.head()

(152488, 73)


,location_id,date,V1,location_name,allbed_mean,allbed_lower,allbed_upper,ICUbed_mean,ICUbed_lower,ICUbed_upper,...,est_infections_mean_p100k_rate,est_infections_lower_p100k_rate,est_infections_upper_p100k_rate,inf_cuml_mean,inf_cuml_upper,inf_cuml_lower,seroprev_mean,seroprev_upper,seroprev_lower,seroprev_data_type
0,1,2020/2/4,48609,Global,14282.96574,14282.96574,14282.96574,5827.528414,5827.528414,5827.528414,...,0.257229,0.216345,0.307828,19902.99479,23818.10353,16739.59124,0.000033,0.000044,0.000024,projected
1,1,2020/2/5,48610,Global,15571.17255,15571.17255,15571.17255,6217.948134,6217.948134,6217.948134,...,0.248723,0.209659,0.296248,39147.82713,46740.16291,32961.88382,0.000037,0.000047,0.000028,projected
2,1,2020/2/6,48611,Global,16762.15309,16762.15309,16762.15309,6559.103608,6559.103608,6559.103608,...,0.240485,0.203118,0.284973,57755.26436,68789.85954,48678.06379,0.000040,0.000051,0.000031,projected
3,1,2020/2/7,48612,Global,17837.85508,17837.85508,17837.85508,6845.497859,6845.497859,6845.497859,...,0.232862,0.196742,0.274455,75772.87967,90025.71510,63900.89907,0.000043,0.000054,0.000034,projected
4,1,2020/2/8,48613,Global,18776.75540,18776.75540,18776.75540,7071.639802,7071.639802,7071.639802,...,0.225968,0.192549,0.266491,93257.07859,110645.38530,78799.28392,0.000046,0.000057,0.000037,projected


In [22]:
df_hospital.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152488 entries, 0 to 152487
Data columns (total 73 columns):
location_id                         152488 non-null int64
date                                152488 non-null object
V1                                  152488 non-null int64
location_name                       152488 non-null object
allbed_mean                         152488 non-null float64
allbed_lower                        152488 non-null float64
allbed_upper                        152488 non-null float64
ICUbed_mean                         152488 non-null float64
ICUbed_lower                        152488 non-null float64
ICUbed_upper                        152488 non-null float64
InvVen_mean                         152488 non-null float64
InvVen_lower                        152488 non-null float64
InvVen_upper                        152488 non-null float64
admis_mean                          152488 non-null float64
admis_lower                         152488 non-null flo

In [28]:
### Since the dataset includes all regions worldwide, need to select us states first
us_states = list(us_state_abbrev.values())
df_us_hospital = df_hospital[df_hospital['location_name'] == us_states[0]]
for i in range(1, len(us_states)):
    name = us_states[i]
    data = df_hospital[df_hospital['location_name'] == name]
    print(name, data.shape[0])
    df_us_hospital = df_us_hospital.append(data)
print(df_us_hospital.shape)

Alabama 392
Arkansas 392
American Samoa 0
Arizona 392
California 392
Colorado 392
Connecticut 392
District of Columbia 392
Delaware 392
Florida 392
Georgia 392
Guam 392
Hawaii 392
Iowa 392
Idaho 392
Illinois 392
Indiana 392
Kansas 392
Kentucky 392
Louisiana 392
Massachusetts 392
Maryland 392
Maine 392
Michigan 392
Minnesota 392
Missouri 392
Northern Mariana Islands 0
Mississippi 392
Montana 392
North Carolina 392
North Dakota 392
Nebraska 392
New Hampshire 392
New Jersey 392
New Mexico 392
Nevada 392
New York 392
Ohio 392
Oklahoma 392
Oregon 392
Pennsylvania 392
Puerto Rico 392
Rhode Island 392
South Carolina 392
South Dakota 392
Tennessee 392
Texas 392
Utah 392
Virginia 392
Virgin Islands 0
Vermont 392
Washington 392
Wisconsin 392
West Virginia 392
Wyoming 392
(20776, 73)


#### Conclusion:
- this dataset doesn't have information for American Samoa, Northern Mariana Islands, and Virgin Islands
- each state covers 392 days

### Preprocess certain columns

In [29]:
datetime = pd.to_datetime(df_us_hospital.iloc[:, 1], format='%Y-%m-%d')
df_us_hospital.insert(2, 'datetime', datetime)
df_us_hospital.drop(['date'], axis=1, inplace=True)
df_us_hospital.head()

,location_id,datetime,V1,location_name,allbed_mean,allbed_lower,allbed_upper,ICUbed_mean,ICUbed_lower,ICUbed_upper,...,est_infections_mean_p100k_rate,est_infections_lower_p100k_rate,est_infections_upper_p100k_rate,inf_cuml_mean,inf_cuml_upper,inf_cuml_lower,seroprev_mean,seroprev_upper,seroprev_lower,seroprev_data_type
70168,524,2020-02-04,2353,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,projected
70169,524,2020-02-05,2354,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,projected
70170,524,2020-02-06,2355,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,projected
70171,524,2020-02-07,2356,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,projected
70172,524,2020-02-08,2357,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,projected


In [30]:
groups = df_us_hospital.groupby('datetime')
dates = []
for name, group in groups:
    dates.append(name)
dates.sort()
print(dates[:1], dates[-1:])

[Timestamp('2020-02-04 00:00:00')] [Timestamp('2021-03-01 00:00:00')]


Hence this dataset covers from 2020-02-04 to 2021-03-01 for every state in U.S. (data with dates starting from the access date is projected)

In [31]:
## Since the first dataset ends at 11/2, we have to use data before 11/3 from this dataset as well
df_hospital_clean = df_us_hospital[df_us_hospital['datetime']<'2020-11-03']
df_hospital_clean.shape

(14469, 73)

In [32]:
null_by_state_hospital = df_hospital_clean.groupby('location_name').apply(lambda x: (x==0).sum())
null_by_state_hospital

,location_id,datetime,V1,location_name,allbed_mean,allbed_lower,allbed_upper,ICUbed_mean,ICUbed_lower,ICUbed_upper,...,est_infections_mean_p100k_rate,est_infections_lower_p100k_rate,est_infections_upper_p100k_rate,inf_cuml_mean,inf_cuml_upper,inf_cuml_lower,seroprev_mean,seroprev_upper,seroprev_lower,seroprev_data_type
location_name,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,0,0,33,33,33,33,33,33,...,18,22,18,18,18,22,30,30,34,0
Alaska,0,0,0,0,26,26,26,26,26,26,...,11,15,11,11,11,15,23,23,27,0
Arizona,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Arkansas,0,0,0,0,30,30,30,30,30,30,...,15,19,15,15,15,19,27,27,31,0
California,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Colorado,0,0,0,0,26,26,26,26,26,26,...,11,15,11,11,11,15,23,23,27,0
Connecticut,0,0,0,0,27,27,27,27,27,27,...,12,16,12,12,12,16,24,24,28,0
Delaware,0,0,0,0,52,52,52,52,52,52,...,37,41,37,37,37,41,49,49,53,0
District of Columbia,0,0,0,0,28,28,28,28,28,28,...,13,17,13,13,13,17,25,25,29,0


In [33]:
## select columns with relatively acceptable null
features = ['datetime', 'location_name', 'allbed_mean', 'ICUbed_mean', 'InvVen_mean', 'admis_mean', 'newICU_mean', 
 'deaths_mean', 'totdea_mean', 'mobility_composite', 'total_tests', 'confirmed_infections', 'total_pop']
df_hospital_clean = df_hospital_clean.loc[:, features]
print(df_hospital_clean.shape)
df_hospital_clean.head()

(14469, 13)


,datetime,location_name,allbed_mean,ICUbed_mean,InvVen_mean,admis_mean,newICU_mean,deaths_mean,totdea_mean,mobility_composite,total_tests,confirmed_infections,total_pop
70168,2020-02-04,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,788027.5317
70169,2020-02-05,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,788027.5317
70170,2020-02-06,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,788027.5317
70171,2020-02-07,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,788027.5317
70172,2020-02-08,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.759226,NaN,NaN,788027.5317


# Merge Dataset 1 & 2
#### Until now, we have:
- df_clean: from 1/22 to 11/2; 56 states; 10 columns
- df_hospital_clean: from 2/4 to 11/2; 53 states; 13 columns

In [73]:
##  According to previous observations, we notice that columns in early months have more null values. 
## Hence we chose to use data starting from 3/27

In [34]:
df_clean  = df_clean[df_clean['datetime']>'2020-03-26']
df_hospital_clean = df_hospital_clean[df_hospital_clean['datetime']>'2020-03-26']

In [35]:
## df_hospital_clean doesn't have data for American Samoa, Northern Mariana Islands, Virgin Islands
## Hence we dropped data for the three regions in df_clean
df_clean = df_clean.drop(df_clean[df_clean['location_name']=='American Samoa'].index, axis=0)
df_clean = df_clean.drop(df_clean[df_clean['location_name']=='Northern Mariana Islands'].index, axis=0)
df_clean = df_clean.drop(df_clean[df_clean['location_name']=='Virgin Islands'].index, axis=0)

In [36]:
print(df_clean.shape)
print(df_hospital_clean.shape)

(11713, 10)
(11713, 13)


In [40]:
df_merge = pd.merge(df_clean, df_hospital_clean, on=['datetime', 'location_name'])
print(df_merge.shape)

(11713, 21)


In [41]:
df_merge.drop(['hospitalizedCurrently', 'inIcuCurrently', 'onVentilatorCurrently', 'recovered'], axis=1, inplace=True)
df_merge

,datetime,location_name,death,negative,positive,totalTestResults,allbed_mean,ICUbed_mean,InvVen_mean,admis_mean,newICU_mean,deaths_mean,totdea_mean,mobility_composite,total_tests,confirmed_infections,total_pop
0,2020-11-02,Alaska,84.0,603113.0,17057.0,620170.0,96.400137,25.956058,11.018175,4.366849,1.599392,1.0,84.0,-18.182785,64.164058,352.0,7.880275e+05
1,2020-11-02,Alabama,2973.0,1190436.0,194892.0,1356420.0,985.931400,265.465311,112.688274,35.241762,17.716837,0.0,2973.0,-8.537036,76.536974,907.0,4.977688e+06
2,2020-11-02,Arkansas,1985.0,1258726.0,113641.0,1363429.0,645.171493,173.714572,73.740690,80.298764,32.241877,27.0,1985.0,-12.654483,97.087762,584.0,3.057349e+06
3,2020-11-02,Arizona,5982.0,1550382.0,248139.0,1792602.0,1058.106450,284.898684,120.937613,66.358930,28.997950,3.0,5982.0,-22.250686,115.791951,666.0,7.249680e+06
4,2020-11-02,California,17672.0,17981873.0,930628.0,18912501.0,3267.641088,789.854105,335.287859,120.673427,51.708698,18.0,17689.0,-32.546875,1365.298154,4533.0,3.987203e+07
5,2020-11-02,Colorado,2105.0,1133557.0,109910.0,2034795.0,834.881033,224.794497,95.423782,32.215485,16.348864,4.0,2292.0,-28.245814,119.001898,2237.0,5.401063e+06
6,2020-11-02,Connecticut,4627.0,2294136.0,73858.0,2367994.0,345.745415,93.093104,39.517409,16.034186,9.281827,11.0,4627.0,-25.379137,244.489556,2651.0,3.693747e+06
7,2020-11-02,District of Columbia,647.0,510736.0,17438.0,528174.0,96.234181,25.911374,10.999207,3.832849,1.375642,1.0,647.0,-40.694680,43.256851,69.0,6.502245e+05
8,2020-11-02,Delaware,710.0,329359.0,25311.0,565983.0,104.266968,25.427501,10.793807,3.916328,1.971511,2.0,712.0,-27.096459,16.901942,115.0,9.750952e+05
9,2020-11-02,Florida,17043.0,5399014.0,801371.0,10126764.0,2405.420042,647.667353,274.930520,149.848353,64.827452,45.0,16834.0,-22.753967,313.344108,4651.0,2.117489e+07


In [44]:
#null_by_state = df_merge.groupby('location_name').apply(lambda x: x.isnull().sum())
#null_by_state

In [45]:
df_merge.to_csv('covid_us_merge.csv', index=False)

# Dataset 3. Mobility data
 - this dataset is downloaded from [a github dataset](https://github.com/GeoDS/COVID19USFlows)
 - we have chosen to download data from 2020-03-27 to 2020-11-02
 - we didn't upload this dataset on GitHub because its size reached maximum

### Dataset Structure & Preprocessing

In [79]:
df_mobility = pd.read_csv('daily_state2state.csv')
print(df_mobility.shape)
df_mobility.head()

(589813, 9)


,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date_range,visitor_flows,pop_flows
0,1,1,-86.844521,32.75688,-86.844521,32.756880,2020/3/27,806784,9782825
1,1,2,-86.844521,32.75688,-151.250549,63.788469,2020/3/27,19,230
2,1,4,-86.844521,32.75688,-111.664460,34.293095,2020/3/27,199,2413
3,1,5,-86.844521,32.75688,-92.439237,34.899772,2020/3/27,470,5699
4,1,6,-86.844521,32.75688,-119.663846,37.215308,2020/3/27,479,5808


#### Conclusion:
- The dataset uses longitude and latitude to represent different states
- For each date, one row records visitor_flows and pop_flows move from one place to another destination
- We want to calculate the total visitor flows and total population flows for every state on each date

In [80]:
datetime = pd.to_datetime(df_mobility.iloc[:, 6], format='%Y-%m-%d')
df_mobility.insert(0, 'datetime', datetime)
df_mobility.drop(['date_range'], axis=1, inplace=True)
df_mobility.head()

,datetime,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,visitor_flows,pop_flows
0,2020-03-27,1,1,-86.844521,32.75688,-86.844521,32.756880,806784,9782825
1,2020-03-27,1,2,-86.844521,32.75688,-151.250549,63.788469,19,230
2,2020-03-27,1,4,-86.844521,32.75688,-111.664460,34.293095,199,2413
3,2020-03-27,1,5,-86.844521,32.75688,-92.439237,34.899772,470,5699
4,2020-03-27,1,6,-86.844521,32.75688,-119.663846,37.215308,479,5808


In [81]:
df_mobility[df_mobility['geoid_o'] == 1]['geoid_d']

0          1
1          2
2          4
3          5
4          6
5          8
6          9
7         10
8         11
9         12
10        13
11        15
12        16
13        17
14        18
15        19
16        20
17        21
18        22
19        23
20        24
21        25
22        26
23        27
24        28
25        29
26        30
27        31
28        32
29        33
          ..
587180    25
587181    26
587182    27
587183    28
587184    29
587185    30
587186    31
587187    32
587188    33
587189    34
587190    35
587191    36
587192    37
587193    38
587194    39
587195    40
587196    41
587197    42
587198    45
587199    46
587200    47
587201    48
587202    49
587203    50
587204    51
587205    53
587206    54
587207    55
587208    56
587209    72
Name: geoid_d, Length: 11491, dtype: int64

In [82]:
## we noticed that the region geoid ==72 has a problematic latitude and longitude, which cannot be recognized on the map
## hence we dropped rows which contains geoid == 72
df_mobility.drop(df_mobility[df_mobility['geoid_o']==72].index, axis=0, inplace=True)
df_mobility.drop(df_mobility[df_mobility['geoid_d']==72].index, axis=0, inplace=True)

### Aggregate visitor flows and pop flows of each state on each date

In [83]:
result = df_mobility.groupby(['datetime', 'geoid_o']).agg({'visitor_flows': 'sum', 'pop_flows': 'sum'})
df_outflow = result.reset_index()
df_outflow.rename(columns={'geoid_o': 'geoid', 'visitor_flows': 'visitor_outflow', 'pop_flows':'pop_outflow'}, inplace=True)
print(df_outflow.shape)
df_outflow.head()

(11271, 4)


,datetime,geoid,visitor_outflow,pop_outflow
0,2020-03-27,1,851538,10325474
1,2020-03-27,2,40167,1081863
2,2020-03-27,4,603781,11941596
3,2020-03-27,5,446011,6067115
4,2020-03-27,6,2377377,65100117


In [84]:
result2 = df_mobility.groupby(['datetime', 'geoid_d']).agg({'visitor_flows': 'sum', 'pop_flows': 'sum'})
df_inflow = result2.reset_index()
df_inflow.rename(columns={'geoid_d': 'geoid', 'visitor_flows': 'visitor_inflow', 'pop_flows':'pop_inflow'}, inplace=True)
print(df_inflow.shape)
df_inflow.head()

(11271, 4)


,datetime,geoid,visitor_inflow,pop_inflow
0,2020-03-27,1,843234,10367598
1,2020-03-27,2,40642,1082693
2,2020-03-27,4,602943,11967096
3,2020-03-27,5,445254,6113562
4,2020-03-27,6,2365173,64430714


In [85]:
df_mob_merge = pd.merge(df_outflow, df_inflow, on=['datetime', 'geoid'])
df_mob_merge.head()

,datetime,geoid,visitor_outflow,pop_outflow,visitor_inflow,pop_inflow
0,2020-03-27,1,851538,10325474,843234,10367598
1,2020-03-27,2,40167,1081863,40642,1082693
2,2020-03-27,4,603781,11941596,602943,11967096
3,2020-03-27,5,446011,6067115,445254,6113562
4,2020-03-27,6,2377377,65100117,2365173,64430714


### Match state names to latitude & longitude

In [86]:
df_mobility.groupby(['geoid_o', 'lat_o', 'lng_o']).groups.keys()

dict_keys([(1, 32.75687994, -86.844521), (2, 63.74298902, -151.5934219), (2, 63.78846948, -151.25054880000002), (4, 34.29309519, -111.66446029999999), (5, 34.89977242, -92.43923686), (6, 37.21530826, -119.6638459), (8, 38.9985316, -105.5478211), (9, 41.57516415, -72.73825768), (10, 38.99497529, -75.45249263), (11, 38.90477389, -77.01629090000002), (12, 28.47705841, -82.46641839), (13, 32.63861711, -83.42714021), (15, 20.9951112, -158.1099738), (16, 44.38905509, -114.65941399999998), (17, 40.12420083, -89.14863899), (18, 39.91986962, -86.28183839), (19, 42.07464833, -93.50009012), (20, 38.48472707, -98.3801554), (21, 37.52661417, -85.29055223), (22, 30.909072899999998, -91.81423318), (23, 45.27432853, -69.20275986), (24, 38.94649396, -76.68717734), (25, 42.16009327, -71.50397204), (26, 44.874773600000005, -85.73095291), (27, 46.34911038, -94.1983056), (28, 32.71289227, -89.65335941), (29, 38.36763044, -92.4774252), (30, 47.03342111, -109.64520700000001), (31, 41.52715113, -99.81085586),

In [87]:
## match state abbreviation with geoid
geo_dict = {1: 'AL', 2:'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 
        11: 'DC', 12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 
       21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 
       31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK',
       41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 
       51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'}   ## 51 regions

In [88]:
location_name = df_mob_merge.geoid.apply(lambda x: us_state_abbrev[geo_dict[x]])
df_mob_merge.insert(1, 'location_name', location_name)
df_mob_merge.drop(['geoid'], axis=1, inplace=True)
df_mob_merge.head()

,datetime,location_name,visitor_outflow,pop_outflow,visitor_inflow,pop_inflow
0,2020-03-27,Alabama,851538,10325474,843234,10367598
1,2020-03-27,Alaska,40167,1081863,40642,1082693
2,2020-03-27,Arizona,603781,11941596,602943,11967096
3,2020-03-27,Arkansas,446011,6067115,445254,6113562
4,2020-03-27,California,2377377,65100117,2365173,64430714


In [89]:
print(df_mob_merge.shape)

(11271, 6)


In [93]:
df_mob_merge.to_csv('mobility_us_merge.csv', index=False)

# Merge Dataset 1, 2, 3
#### Until now, we have:
- dataset 1&2 merged together: covers from 3/27 to 11/2, 53 states
- dataset 3 (mobility data): covers from 3/27 to 11/2, 51 states (Guam, Puerto Rico)

In [91]:
## this datasheet has merged dataset 1 & 2
df = pd.read_csv('covid_us_merge.csv')
print(df.shape)
df.head()

(11713, 17)


,datetime,location_name,death,negative,positive,totalTestResults,allbed_mean,ICUbed_mean,InvVen_mean,admis_mean,newICU_mean,deaths_mean,totdea_mean,mobility_composite,total_tests,confirmed_infections,total_pop
0,2020-11-02,Alaska,84.0,603113.0,17057.0,620170.0,96.400137,25.956058,11.018175,4.366849,1.599392,1.0,84.0,-18.182785,64.164058,352.0,7.880275e+05
1,2020-11-02,Alabama,2973.0,1190436.0,194892.0,1356420.0,985.931400,265.465311,112.688274,35.241762,17.716837,0.0,2973.0,-8.537036,76.536974,907.0,4.977688e+06
2,2020-11-02,Arkansas,1985.0,1258726.0,113641.0,1363429.0,645.171493,173.714572,73.740690,80.298764,32.241877,27.0,1985.0,-12.654483,97.087762,584.0,3.057349e+06
3,2020-11-02,Arizona,5982.0,1550382.0,248139.0,1792602.0,1058.106450,284.898684,120.937613,66.358930,28.997950,3.0,5982.0,-22.250686,115.791951,666.0,7.249680e+06
4,2020-11-02,California,17672.0,17981873.0,930628.0,18912501.0,3267.641088,789.854105,335.287859,120.673428,51.708698,18.0,17689.0,-32.546875,1365.298154,4533.0,3.987203e+07


In [94]:
df2 = pd.read_csv('mobility_us_merge.csv')
print(df2.shape)
df2.head()

(11271, 6)


,datetime,location_name,visitor_outflow,pop_outflow,visitor_inflow,pop_inflow
0,2020-03-27,Alabama,851538,10325474,843234,10367598
1,2020-03-27,Alaska,40167,1081863,40642,1082693
2,2020-03-27,Arizona,603781,11941596,602943,11967096
3,2020-03-27,Arkansas,446011,6067115,445254,6113562
4,2020-03-27,California,2377377,65100117,2365173,64430714


In [95]:
df_all = pd.merge(df2, df, on=['datetime', 'location_name'])
print(df_all.shape)
df_all.head()

(11271, 21)


,datetime,location_name,visitor_outflow,pop_outflow,visitor_inflow,pop_inflow,death,negative,positive,totalTestResults,...,ICUbed_mean,InvVen_mean,admis_mean,newICU_mean,deaths_mean,totdea_mean,mobility_composite,total_tests,confirmed_infections,total_pop
0,2020-03-27,Alabama,851538,10325474,843234,10367598,3.0,4184.0,587.0,4771.0,...,43.517784,30.655464,8.480432,4.133427,3.0,4.0,-30.616021,6.058722,71.0,4.977688e+06
1,2020-03-27,Alaska,40167,1081863,40642,1082693,1.0,2319.0,69.0,2388.0,...,35.580906,25.064447,1.920386,0.654095,0.0,1.0,-46.856247,3.464309,10.0,7.880275e+05
2,2020-03-27,Arizona,603781,11941596,602943,11967096,13.0,7455.0,736.0,8191.0,...,39.294831,27.680667,13.369623,5.642218,5.0,13.0,-42.911514,16.092132,158.0,7.249680e+06
3,2020-03-27,Arkansas,446011,6067115,445254,6113562,3.0,1545.0,381.0,1926.0,...,14.464876,10.189569,4.690174,1.716415,1.0,3.0,-32.910349,6.228745,45.0,3.057349e+06
4,2020-03-27,California,2377377,65100117,2365173,64430714,78.0,17380.0,3879.0,21259.0,...,417.665987,294.218673,86.191841,33.690555,13.0,94.0,-50.200762,64.754388,758.0,3.987203e+07


In [103]:
state_names = list(df_all.groupby('location_name').groups.keys())
len(state_names)

51

In [104]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11271 entries, 0 to 11270
Data columns (total 21 columns):
datetime                11271 non-null object
location_name           11271 non-null object
visitor_outflow         11271 non-null int64
pop_outflow             11271 non-null int64
visitor_inflow          11271 non-null int64
pop_inflow              11271 non-null int64
death                   11263 non-null float64
negative                11271 non-null float64
positive                11271 non-null float64
totalTestResults        11271 non-null float64
allbed_mean             11271 non-null float64
ICUbed_mean             11271 non-null float64
InvVen_mean             11271 non-null float64
admis_mean              11271 non-null float64
newICU_mean             11271 non-null float64
deaths_mean             11064 non-null float64
totdea_mean             11064 non-null float64
mobility_composite      11271 non-null float64
total_tests             11271 non-null float64
confirme

In [105]:
df_all.drop(['deaths_mean', 'totdea_mean', 'confirmed_infections'], axis=1, inplace=True)

In [106]:
df_all = df_all[df_all['datetime']>='2020-04-01']

In [109]:
print(df_all.shape)
#df_all.groupby(['location_name']).apply(lambda x: x.isnull().sum())

(11016, 18)


In [110]:
#df_all.groupby(['location_name']).apply(lambda x: (x==0).sum())

In [111]:
df_all.to_csv('all_us_data.csv', index=False)

*We have uploaded the final version of dataset to GitHub*